<a href="https://colab.research.google.com/github/ProshkinNV/Image/blob/main/GitHub_Image_Stanford_Dog_%E2%84%962.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import shutil

from tensorflow import data as tf_data
import os
import keras
from keras import layers
from keras.applications import EfficientNetV2B0

In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/stanford_dogs.zip

In [ ]:
# Разархивируем датасета во временную папку 'temp'
!unzip -qo "stanford_dogs" -d ./dataset

# Папка с папками картинок, рассортированных по категориям
IMAGE_PATH = './dataset/'

In [ ]:
num_skipped = 0 # счетчик поврежденных файлов
for folder_name in os.listdir(IMAGE_PATH): # перебираем папки
    folder_path = os.path.join(IMAGE_PATH, folder_name) # склеиваем путь
    for fname in os.listdir(folder_path): # получаем список файлов в папке
        fpath = os.path.join(folder_path, fname) # получаем путь до файла
        try:
            fobj = open(fpath, "rb") # пытаемся открыть файл для бинарного чтения (rb)
            is_jfif = b"JFIF" in fobj.peek(10) # получаем первые 10 байт из файла и ищем в них бинарный вариант строки JFIF
        finally:
            fobj.close() # Закрываем файл

        if not is_jfif: # Если не нашли JFIF строку
            # Увеличиваем счетчик
            num_skipped += 1
            # Удаляем поврежденное изображение
            os.remove(fpath)

print(f"Удалено изображений: {num_skipped}")

In [ ]:
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    IMAGE_PATH, # путь к папке с данными
    validation_split=0.2, # отщепляем 20% на проверочную выборку
    subset="both", # указываем, что необходимо вернуть кортеж из обучающей и проверочной выборок ("training", "validation" или "both")
    seed=42,  # воспроизводимость результата генерации (результаты с одинаковым числом - одинаковы),
    shuffle=True, # перемешиваем датасет
    image_size=IMAGE_SIZE, # размер генерируемых изображений
    batch_size=BATCH_SIZE, # размер мини-батча
)

In [ ]:
# Определяем список имен классов
CLASS_LIST = sorted(os.listdir(IMAGE_PATH))

# Определяем количества классов
CLASS_COUNT = len(CLASS_LIST)

# Вывод результата
print(f'Количество классов: {CLASS_COUNT}')
print(f'Метки классов: {CLASS_LIST}')

In [ ]:
RAND_CLASS = []
random.shuffle(CLASS_LIST)

i = 0
while (i < 10):
  RAND_CLASS.append(CLASS_LIST[i])
  i+= 1

print(RAND_CLASS)

['Samoyed', 'Border_collie', 'Collie', 'Afghan_hound', 'English_springer', 'Bouvier_des_Flandres', 'American_Staffordshire_terrier', 'Appenzeller', 'Scottish_deerhound', 'Border_terrier']


In [ ]:
print("Текущая деректория:", os.getcwd())

Текущая деректория: /content


In [ ]:
OLD_PATH = './dataset/'
NEW_PATH = './folder_RAND_CLASS/'

IMAGE_SIZE = (224, 224) # Размер определен выбором модели
BATCH_SIZE = 128

if not os.path.exists('folder_RAND_CLASS'):
  os.mkdir("folder_RAND_CLASS")

In [ ]:
for folder in RAND_CLASS:
  old_temp_path = OLD_PATH + folder
  new_temp_path = NEW_PATH + folder
  shutil.copytree(old_temp_path, new_temp_path)

**Создание датасэта**

In [ ]:
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    NEW_PATH, # путь к папке с данными
    validation_split=0.2, # отщепляем 20% на проверочную выборку
    subset="both", # указываем, что необходимо вернуть кортеж из обучающей и проверочной выборок ("training", "validation" или "both")
    seed=42,  # воспроизводимость результата генерации (результаты с одинаковым числом - одинаковы),
    shuffle=True, # перемешиваем датасет
    image_size=IMAGE_SIZE, # размер генерируемых изображений
    batch_size=BATCH_SIZE, # размер мини-батча
)

Found 1788 files belonging to 10 classes.
Using 1431 files for training.
Using 357 files for validation.


**Определяем метки**

In [ ]:
# Определяем список имен классов
CLASS_LIST_NEW = sorted(RAND_CLASS)

# Определяем количества классов
CLASS_COUNT_NEW = len(RAND_CLASS)

# Вывод результата
print(f'Количество классов: {CLASS_COUNT_NEW}')
print(f'Метки классов: {CLASS_LIST_NEW}')

Количество классов: 10
Метки классов: ['Afghan_hound', 'American_Staffordshire_terrier', 'Appenzeller', 'Border_collie', 'Border_terrier', 'Bouvier_des_Flandres', 'Collie', 'English_springer', 'Samoyed', 'Scottish_deerhound']


**Аугментация изображений**

In [ ]:
img_augmentation_layers = [
    layers.RandomRotation(factor=0.15), # Вращаем изображение в пределах 15%
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1), # Сдвиг на 10% по вертикали и горизонтали
    layers.RandomFlip(), # Отражение по вертикали и горизонтали
    layers.RandomContrast(factor=0.1), # Изменяем контрастность на 10%
]


def img_augmentation(images):
    # Слои - это функции, которые мы последовательно применяем к входным данным
    for layer in img_augmentation_layers:
        images = layer(images)
    return images

**Предварительная обработка данных**

In [ ]:
# Применяем `img_augmentation` к обучающей выборке
train_ds = train_ds.map(
    lambda img, label: (img_augmentation(img), keras.ops.one_hot(label, CLASS_COUNT_NEW)), # One-hot кодирование
    num_parallel_calls=tf_data.AUTOTUNE, # число потоков для обработки в map (автонастройка зависит от возможностей процессора)
)

val_ds = val_ds.map(
    lambda img, label: (img, keras.ops.one_hot(label, CLASS_COUNT_NEW)), # One-hot кодирование
    num_parallel_calls=tf_data.AUTOTUNE, # число потоков для обработки в map (автонастройка зависит от возможностей процессора)
)

# Предварительная выборка примеров в память GPU или оперативную память
# Помогает максимально эффективно использовать графический процессор
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

**Построение модели**

In [ ]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    model = EfficientNetV2B0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Заморозка предобученных весов
    model.trainable = False

    # Замена верхних слоев
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Компилируем
    model = keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"]
    )
    return model

**Обучаем модель**

In [ ]:
epochs = 5

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath = 'best_model_pretrain.keras',
                             monitor = 'val_acc',
                             save_best_only = True,
                             mode = 'max',
                             verbose = 0)
]

model = build_model(num_classes=CLASS_COUNT_NEW)


history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)

Epoch 1/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 130s 6s/step - acc: 0.5935 - loss: 1.4035 - val_acc: 0.9048 - val_loss: 0.2172
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - acc: 0.8532 - loss: 0.6086 - val_acc: 0.9580 - val_loss: 0.1416
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - acc: 0.8748 - loss: 0.5002 - val_acc: 0.9160 - val_loss: 0.1514
Epoch 4/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - acc: 0.8635 - loss: 0.5338 - val_acc: 0.9608 - val_loss: 0.1077
Epoch 5/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - acc: 0.8845 - loss: 0.5414 - val_acc: 0.9552 - val_loss: 0.1135


Точность на обучающей выборка достигла 88%

Точночть на валидационной выборке достигла 96%